In [1]:
from muselsl import *
import numpy as np
import matplotlib.pyplot as plt
from pylsl import StreamInlet, resolve_byprop
import json
import warnings

In [5]:
# -*- coding: utf-8 -*-
"""
Estimate Relaxation from Band Powers
This example shows how to buffer, epoch, and transform EEG data from a single
electrode into values for each of the classic frequencies (e.g. alpha, beta, theta)
Furthermore, it shows how ratios of the band powers can be used to estimate
mental state for neurofeedback.
The neurofeedback protocols described here are inspired by
*Neurofeedback: A Comprehensive Review on System Design, Methodology and Clinical Applications* by Marzbani et. al
Adapted from https://github.com/NeuroTechX/bci-workshop
"""

import numpy as np  # Module that simplifies computations on matrices
import matplotlib.pyplot as plt  # Module used for plotting
from pylsl import StreamInlet, resolve_byprop  # Module to receive EEG data
import utils  # Our own utility functions
from process_data import ChannelDataProcessor, Metrics


# Handy little enum to make code more readable


class Band:
    Delta = 0
    Theta = 1
    Alpha = 2
    Beta = 3


# We are working with 4 channels (Billy) [0], [1], [2], [3] as 4 index_channel values
NUM_CHANNELS = 5

# Length of the EEG data buffer (in seconds)
# This buffer will hold last n seconds of data and be used for calculations
BUFFER_LENGTH = 5

# Length of the epochs used to compute the FFT (in seconds)
EPOCH_LENGTH = 1

# Amount of overlap between two consecutive epochs (in seconds)
OVERLAP_LENGTH = 0.8

# Amount to 'shift' the start of each next consecutive epoch
SHIFT_LENGTH = EPOCH_LENGTH - OVERLAP_LENGTH


def _acquire_eeg_data(_inlet):
    """ Pull EEG data from inlet and return. 

    :return: tuple: _eeg_data, _timestamp
    """
    _eeg_data, _timestamp = _inlet.pull_chunk(
        timeout=1, max_samples=int(SHIFT_LENGTH * fs))
    return _eeg_data, _timestamp

def _acquire_eeg_data_mock():
    """ Get eeg_data and timestamp from json for testing purposes. 

    :return: tuple: _eeg_data, _timestamp
    """
    json_file = open("to_send.json")
    k = json.loads(json_file.read())
    _eeg_data = k[0]["eeg_data"]
    _timestamp = k[0]["timestamp"]
    return _eeg_data, _timestamp

if __name__ == "__main__":

    # """ 1. CONNECT TO EEG STREAM """
    # 
    # Search for active LSL streams
    print('Looking for an EEG stream...')
    streams = resolve_byprop('type', 'EEG', timeout=2)
    if len(streams) == 0:
        raise RuntimeError('Can\'t find EEG stream.')
    
    # Set active EEG stream to inlet and apply time correction
    print("Start acquiring data")
    inlet = StreamInlet(streams[0], max_chunklen=12)
    eeg_time_correction = inlet.time_correction()
    
    # Get the stream info and description
    info = inlet.info()
    description = info.desc()
    
    
    # # Get the sampling frequency
    # # This is an important value that represents how many EEG data points are
    # # collected in a second. This influences our frequency band calculation.
    # # for the Muse 2016, this should always be 256
    
    def get_fs(_info):
        return int(_info.nominal_srate())
    
    def get_fs_mock():
        warnings.warn("Using mock fs value: 256. Make sure that you are using get_fs outside of testing. ")
        return 256  # For testing purposes
    
    fs = get_fs(info)
#     fs = get_fs_mock() 

    """ 3. GET DATA """

    # The try/except structure allows to quit the while loop by aborting the
    # script with <Ctrl-C>
    print('Press Ctrl-C in the console to break the while loop.')

    try:
        # The following loop acquires data, computes band powers, and calculates neurofeedback metrics based on those band powers
        while True:

            """ 3.1 ACQUIRE DATA """
            
            # Obtain EEG data from the LSL stream
            eeg_data, timestamp = _acquire_eeg_data(inlet) 
#             eeg_data, timestamp = _acquire_eeg_data_mock()
            
            c = ChannelDataProcessor(buffer_length=BUFFER_LENGTH, 
                                     epoch_length=EPOCH_LENGTH, 
                                     overlap_length=OVERLAP_LENGTH, 
                                     shift_length=SHIFT_LENGTH, fs=fs, band_cls=Band)
            
            c.feed_new_data(eeg_data=eeg_data)          # Feed new data generated in the epoch
            metrics = np.zeros(NUM_CHANNELS)                       

            for i in range(NUM_CHANNELS):               # Iterate through all separate channels

                # Record channel smooth band power
                csbp = c.get_channel_smooth_band_powers(i)

                # Run calculations on csbp to obtain desired metrics
                alpha_metric = Metrics.alpha_protocol(csbp, Band)
                print("Alpha metric: {}".format(alpha_metric))
                
#                 index_channel = [i]
                metrics[i] = alpha_metric

            print("alpha metric for 4 sensors are separately: {}".format(metrics))

#             break  # so that the screen is not flooded when testing 

    except KeyboardInterrupt:
        print('Closing!')


Looking for an EEG stream...
Start acquiring data
Press Ctrl-C in the console to break the while loop.
Delta:  0.4978067257985036  Theta:  -0.004054728356235762  Alpha:  -0.29231720468123396  Beta:  -0.5984947482548568
Alpha Relaxation:  -0.5872102354831474
Alpha metric: -0.5872102354831474
Delta:  1.7063031046795756  Theta:  1.0827922699854315  Alpha:  0.8378200475244301  Beta:  0.5396894430173605
Alpha Relaxation:  0.49101478232483386
Alpha metric: 0.49101478232483386
Delta:  0.48906125656443944  Theta:  0.05158475970044137  Alpha:  0.12678624487748757  Beta:  0.3699827375504558
Alpha Relaxation:  0.25924409913010976
Alpha metric: 0.25924409913010976
Delta:  0.49951357094941706  Theta:  -0.023592276701530964  Alpha:  -0.2513488699201101  Beta:  -0.4179488212051339
Alpha Relaxation:  -0.5031872696519045
Alpha metric: -0.5031872696519045
Delta:  0.42779621213587593  Theta:  -0.11560268823109113  Alpha:  -0.2918212697137174  Beta:  -0.41787497843003957
Alpha Relaxation:  -0.682150195432